In [1]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests

In [2]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [3]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [4]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [5]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [6]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [7]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [8]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        count_str = re.search('of(.*)jobs', temp_str) 
        count = int(count_str.group(1))
    return(count)

In [9]:
max_results = 100
title_name = "data+scientist"
city_name = "San+Francisco"
st_name = "CA"
columns = ["city", "job_title", "company", "location", "salary", "description"]

city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
page = requests.get(city_url)
soup = BeautifulSoup(page.text, "html.parser")
max_counter = extract_count(soup)
counter = 0
print(max_counter)

job_title_list = []
company_list = []
location_list = []
salary_list = []
desc_list = []

for start in range(0, max_results, 10):
    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name + \
               "&start=" + str(start)
    print(city_url)
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")

    job_title_list.extend(extract_job_title(soup))
    company_list.extend(extract_company(soup))
    location_list.extend(extract_location(soup))
    salary_list.extend(extract_salary(soup))
    add_urls = extract_url(soup)
    desc_list.extend(extract_desc(add_urls))
    
    # counter = len(job_title_list)
    # if counter >= max_counter:
    #     break
    
indeed_df = pd.DataFrame(list(zip(job_title_list, company_list, location_list, 
                                  salary_list, desc_list)), 
                         columns = ['job_title', 'company', 'location', 'salary', 'description'])

933
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=0
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=10
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=20
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=30
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=40
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=50
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=60
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=70
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=80
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=90


In [10]:
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [11]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,low_salary,high_salary
0,Data Scientist,Seen by Indeed,None,With one application you can be considered for...,NaN,NaN
1,Senior Data Scientist,Freedom Financial Network,None,WHO WE ARE:\nFreedom Financial Network is a fa...,NaN,NaN
2,Data Scientist,Affinity,None,Affinity is on a mission to help everyone in t...,NaN,NaN
3,Senior Data Scientist - Authorship,"Turnitin, LLC",None,Company Description\n\nTurnitin is your partne...,NaN,NaN
4,Data Scientist,Grammarly,None,The opportunity\n\n\nGrammarly empowers people...,NaN,NaN


In [12]:
def clean_text(text):
    text = text.replace('\\n', ' ')               # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

indeed_df['description'] = indeed_df.apply(lambda x: clean_text(x['description']), axis=1)

indeed_df.head()

,job_title,company,location,description,low_salary,high_salary
0,Data Scientist,Seen by Indeed,None,with one application you can be considered for...,NaN,NaN
1,Senior Data Scientist,Freedom Financial Network,None,who we arefreedom financial network is a famil...,NaN,NaN
2,Data Scientist,Affinity,None,affinity is on a mission to help everyone in t...,NaN,NaN
3,Senior Data Scientist - Authorship,"Turnitin, LLC",None,company descriptionturnitin is your partner in...,NaN,NaN
4,Data Scientist,Grammarly,None,the opportunitygrammarly empowers people to th...,NaN,NaN


In [13]:
indeed_df['description'][0]

'with one application you can be considered for thousands of tech roles from leading companies on seenseen by indeed is a free service that connects you to opportunities that take you further in your careerjoin today to be seen and make your mark in your next tech rolewith just one application well show you off to hundreds of employers once your profile is approvedseen matches you to opportunities based off where youve been and where you want to go in your careertalk to career coaches for free to help you set your best foot forward whether it be during the interview process or during a promotionby creating an account you agree to indeeds terms of service cookie policy and privacy policy'

In [15]:
indeed_df['description'][4]

'the opportunitygrammarly empowers people to thrive and connect whenever and wherever they communicate more than 20 million people around the world use our aipowered writing assistant every day all of this begins with our team collaborating in a valuesdriven and learningoriented environmentwere looking for a data scientist to help us drive decisionmaking find bigger opportunities and improve our product as a consumer internet company we produce huge amounts of data and in this role you will have outsize opportunities to develop the insights to make it actionable you will have broad impact and exposure across grammarly working with team members from our product research marketing engineering and finance teamsgrammarlys engineers and researchers have the freedom to innovate and uncover breakthroughsand in turn influence our product roadmap the complexity of our technical challenges is growing rapidly as we scale our interfaces algorithms and infrastructure read more about our stack or he